In [1]:
%load_ext autoreload
%autoreload 2
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament
)

In [2]:
db.session.close()

In [3]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [4]:
# purge all tables first

for Model in [TournamentModel, BracketModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [5]:
# add all users to users table
users = [
    UserModel(username='TPN', email='TPN@example.com'.lower()), 
    UserModel(username='AngryFalco', email='AngryFalco@example.com'.lower()), 
    UserModel(username='Sunrisebanana', email='Sunrisebanana@example.com'.lower()), 
    UserModel(username='Ptolemy', email='Ptolemy@example.com'.lower()), 
    UserModel(username='Vik', email='Vik@example.com'.lower()), 
    UserModel(username='Kevin', email='Kevin@example.com'.lower()), 
    UserModel(username='Spaceghost', email='Spaceghost@example.com'.lower()), 
    UserModel(username='Burnaby', email='Burnaby@example.com'.lower()), 
]
[db.session.add(u) for u in users]
db.session.commit()

In [6]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [7]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [8]:
btype = BracketTypes.DOUBLE_ELIMINATION
t = Tournament(tuple_list, btype)

In [9]:
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
TO = t.bracket.entrants[0]

t.post_to_db(tournament_name, TO,)

In [16]:
# t.bracket.post_to_db()

In [ ]:
rds = t.bracket.rounds

In [18]:
for r in t.bracket.rounds:
    print(r)

In [12]:
def post_tournament_to_db(t, t_name, TO, db):
    
    def post_bracket_to_db(t_model):
        
        u_models = \
            [UserModel.query.filter_by(username=u).first_or_404() for u \
             in t.bracket.entrants]
        
        b_model = BracketModel(
            bracket_type=t.bracket.bracketType.name,
            users=u_models,
            tournament=t_model,
        )

        db.session.add(b_model)
        db.session.commit()
        
        return 
    
    
    o_id = \
        UserModel.query.filter_by(username=TO).first_or_404().id
    
    t_model = TournamentModel(
        n_entrants = len(t.bracket.entrants),
        name = t_name,
        organizer_id = o_id
    )
    
    db.session.add(t_model)
    db.session.commit()
    
    post_bracket_to_db(t_model)
    
    return 

In [13]:
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
TO = t.bracket.entrants[0]

post_tournament_to_db(t, tournament_name, TO, db)

In [34]:
print(t.bracket.bracketType.name)
print(t.bracket.entrants)

DOUBLE_ELIMINATION
['Kevin', 'Spaceghost', 'Sunrisebanana', 'TPN', 'AngryFalco', 'Vik', 'Burnaby', 'Ptolemy']


In [37]:
u_models = \
    [UserModel.query.filter_by(username=u).first_or_404() for u \
     in t.bracket.entrants]
u_models

[<User Kevin>,
 <User Spaceghost>,
 <User Sunrisebanana>,
 <User TPN>,
 <User AngryFalco>,
 <User Vik>,
 <User Burnaby>,
 <User Ptolemy>]

In [9]:
b = \
    Bracket(
        bracket_type='Winners',
        users=[u for u in User.query.order_by(func.random()).limit(t.n_entrants).all()],
        tournament=t,
    )

db.session.add(b)
db.session.commit()

In [10]:
b

<Winners bracket: tournament 1>

In [11]:
usrs = [u for u in User.query.order_by(func.random()).limit(t.n_entrants).all()]
# usrs = [(usrs[i], usrs[i+1]) for i in range(0, len(usrs)//2)]

In [12]:
usrs[0]

<User Sunrisebanana>

In [13]:
r = \
    Round(
        number=1,
        winners=True,
        bracket=b,
#         matches=matches
    )
r

round 1 in bracket None

In [14]:
db.session.add(r)
db.session.commit()

In [15]:
r.id

1

In [16]:
matches = \
    [Match(
#         users=list((usrs[i], usrs[i+1])),
        user_1 = usrs[i].id,
        user_2 = usrs[i+1].id,
        round_id=r.id
    ) for i in range(0, len(usrs), 2)]
m = matches[0]
m

<match None between 3 and 2>

In [17]:
for m in matches:
    db.session.add(m)

In [18]:
db.session.commit()

In [10]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-18 23:57:34.644749
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-18 23:57:34.645228
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-18 23:57:34.645362
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-18 23:57:34.645478
4,5,Vik,vik@example.com,None,None,2020-02-18 23:57:34.645569
5,6,Kevin,kevin@example.com,None,None,2020-02-18 23:57:34.645661
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-18 23:57:34.645743
7,8,Burnaby,burnaby@example.com,None,None,2020-02-18 23:57:34.645813


In [11]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,4


In [12]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [13]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,4,1
1,6,1
2,1,1
3,3,1
4,7,1
5,8,1
6,2,1
7,5,1


In [14]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id


In [15]:
pd.read_sql_query('select * from match', cnx)

,id,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to


In [25]:
# create some new matches for winners/losers to advance to
matches = \
    [Match(
#         users=list((usrs[i], usrs[i+1])),
#         user_1 = usrs[i].id,
#         user_2 = usrs[i+1].id,
        round_id=r.id
    ) for i in range(2)]
m = matches[0]

In [26]:
db.session.add(m)
db.session.commit()

In [27]:
m

<match 5 between None and None>